<a href="https://colab.research.google.com/github/AutomaticHourglass/SAR1/blob/master/Flevoland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
!pip install pydrive
# !pip install lightgbm
!pip install catboost
!pip install xgboost
!pip install --upgrade --user hmmlearn

Requirement already up-to-date: hmmlearn in /root/.local/lib/python3.6/site-packages (0.2.1)


In [0]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# !cd LightGBM
# !mkdir build ; cd build
# !cmake -DUSE_GPU=1 ..
# # if you have installed NVIDIA CUDA to a customized location, you should specify paths to OpenCL headers and library like the following:
# # cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
# !make -j$(nproc)
# !cd ..

# !apt-get -qq install --no-install-recommends nvidia-375
# # !apt-get -qq install --no-install-recommends nvidia-opencl-icd-375 nvidia-opencl-dev opencl-headers
# # !apt-get -qq install --no-install-recommends git cmake build-essential libboost-dev libboost-system-dev libboost-filesystem-dev
# # !pip install -qq lightgbm --install-option=--gpu

In [0]:
def download_file(file_id, filename):
  import io
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  from googleapiclient.http import MediaIoBaseDownload

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  from googleapiclient.discovery import build
  drive_service = build('drive', 'v3')

  request = drive_service.files().get_media(fileId=file_id)
  fh = io.FileIO(filename, 'wb')
  downloader = MediaIoBaseDownload(fh, request)
  done = False
  while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
    


In [0]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt


thesis_base_folder_id = '0B1XCkiXOusbmX3ZiTF9GaWZfQlU'
flevoland_data_id = '1tKZLH53sfawd9dKnTVA1CADWtC9qXgMO'
flevoland_alim_samat_id = '1Xves_MigWqzC_DpZjRjgbXcGeqdT5twK'

flevoland_data_name = 'flevoland_data.mat'

data_source = 'alim'

if data_source == 'alim':
  download_file(flevoland_alim_samat_id,flevoland_data_name)
  mat = scipy.io.loadmat('flevoland_data.mat')
  data_img = np.real(np.array(mat['T3']))
  label_roi_tr = np.array(mat['tr_label_img'])
  label_roi_ts = np.array(mat['ts_label_img'])
else:
  download_file(flevoland_data_id,flevoland_data_name)
  mat = scipy.io.loadmat('flevoland_data.mat')
  data_img = np.real(np.array(mat['data']))
  label_img = np.array(mat['label'])

Download 100%.


In [0]:


# mat = scipy.io.loadmat('flevoland_data.mat')
# data_img = np.real(np.array(mat['data']))
# data_img = data_img[:,:,[1,2,4]]
# r1 = data_img[:,:,0]/(data_img[:,:,3]+1e-6)
# r2 = data_img[:,:,0]/(data_img[:,:,5]+1e-6)
# r3 = data_img[:,:,3]/(data_img[:,:,5]+1e-6)

# r1 = np.expand_dims(r1,axis=2)
# r2 = np.expand_dims(r2,axis=2)
# r3 = np.expand_dims(r3,axis=2)
# span = data_img[:,:,0] + data_img[:,:,3] + data_img[:,:,5]
# span = np.expand_dims(span,axis=2)

# data_img = np.concatenate((data_img,r1,r2,r3),axis=2)


In [0]:
def extract_patches(data_img, label_img, patch_size = (8,8), stride = 4, border = (3,3)):
  from sklearn.feature_extraction import image

  data_patches = image.extract_patches_2d(data_img,patch_size)
  label_patches = image.extract_patches_2d(label_img,patch_size)

  data_patches = data_patches[::stride]
  label_patches = label_patches[::stride]

  from collections import Counter

  label = np.zeros(label_patches.shape[0])
  cnt = 0

  for patch in label_patches:
    roi_patch = patch[border[0]:-border[0],border[1]:-border[1]]
    c= Counter(patch.flatten())
    x = c.most_common()[0][0]
    label[cnt] = x
    cnt += 1

  ind = label>0
  data_patches = data_patches[ind]
  label_patches = label_patches[ind]
  label -= 1
  label = label[ind]
  
  return data_patches, label

In [0]:
if data_source != 'alim':
  x1 = 200
  x2 = 950
  y1 = 200
  y2 = 600
  label_roi_ts = label_img.copy()
  label_roi_ts[:x1,:] = 0
  label_roi_ts[x2:,:] = 0
  label_roi_ts[:,:y1] = 0
  label_roi_ts[:,y2:] = 0
  label_roi_tr = label_img.copy()
  label_roi_tr[x1:x2,y1:y2] = 0

  label_roi_tr[990:,320:360] = 0
  label_roi_ts[990:,320:360] = label_img[990:,320:360]

  ts_class = np.unique(label_roi_ts.flatten())
  tr_class = np.unique(label_roi_tr.flatten())
  print(ts_class)
  print(tr_class)
  print(list(set(tr_class) & set(ts_class)))
  print(len(list(set(tr_class) & set(ts_class))))
  print(label_img.shape)



  plt.figure(figsize=(8,8))
  plt.imshow(label_img, cmap='jet')
  plt.show()
  plt.figure(figsize=(8,8))
  plt.imshow(label_roi_tr, cmap='jet')
  plt.show()
  plt.figure(figsize=(8,8))
  plt.imshow(label_roi_ts, cmap='jet')
  plt.show()

# Vectorize 1x1 pixel
data_vector_tr = data_img[label_roi_tr>0,:]
data_vector_ts = data_img[label_roi_ts>0,:]
label_vector_tr = label_roi_tr[label_roi_tr>0]
label_vector_ts = label_roi_ts[label_roi_ts>0]

In [0]:
patch_size = (12,12)
stride = 4
border = (2,2)
data_tr, label_tr = extract_patches(data_img,label_roi_tr, patch_size=patch_size, stride=stride, border=border)
data_ts, label_ts = extract_patches(data_img,label_roi_ts, patch_size=patch_size, stride=stride, border=border)

print(data_ts.shape)

from collections import Counter

c_tr = Counter(label_tr)
c_ts = Counter(label_ts)

print(sorted(c_tr.items()))
print(sorted(c_ts.items()))
print(label_tr.shape)
print(label_ts.shape)

# Vectorize nxn tiles
data_tr_vector = data_tr.reshape((data_tr.shape[0],-1))
data_ts_vector = data_ts.reshape((data_ts.shape[0],-1))

# # Data Augmentation
# data_tr = np.concatenate((data_tr,data_tr[:,::-1,:,:],data_tr[:,:,::-1,:],data_tr[:,::-1,::-1,:]),axis=0)
# label_tr = np.concatenate((label_tr,label_tr,label_tr,label_tr),axis=0)

In [0]:
from __future__ import print_function

import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from sklearn import preprocessing

# np.random.seed(1)
# tf.set_random_seed(2)

if data_source == 'alim':
  num_classes = 4
else:
  num_classes = 11

batch_size = 128
epochs = 20
validate_gap = 10

label_tr_cat = keras.utils.to_categorical(label_tr, num_classes)
label_ts_cat = keras.utils.to_categorical(label_ts, num_classes)

model = Sequential()

# model.add(BatchNormalization())
# model.add(Conv2D(32, kernel_size=(5, 5),
#                  activation='relu'))

model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3),
                 activation='relu'))
model.add(Dropout(0.2))

model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3),
                 activation='relu'))
model.add(Dropout(0.2))


# model.add(BatchNormalization())
# model.add(Conv2D(32, kernel_size=(2, 2),
#                  activation='relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(128, kernel_size=(1, 1),
#                  activation='relu'))

# model.add(Dropout(0.1))
model.add(Flatten())

# model.add(Dense(512))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.1))

# model.add(Dense(256))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.15))

# model.add(Dense(128))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.15))

# model.add(Dense(64))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# model.add(BatchNormalization())
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.1))

model.add(BatchNormalization())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adamax(),
              metrics=['accuracy'])

model.fit(data_tr, label_tr_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,validation_data=(data_ts[::validate_gap], label_ts_cat[::validate_gap]))
print(model.summary())

# score = model.evaluate(data_ts, label_ts_cat, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 1165 samples, validate on 16664 samples
Epoch 1/20
1165/1165 [==============================] - 6s 5ms/step - loss: 0.5045 - acc: 0.8369 - val_loss: 1.2133 - val_acc: 0.7622
Epoch 2/20
1165/1165 [==============================] - 2s 1ms/step - loss: 0.0721 - acc: 0.9708 - val_loss: 1.5460 - val_acc: 0.7510
Epoch 3/20
1165/1165 [==============================] - 2s 1ms/step - loss: 0.0594 - acc: 0.9803 - val_loss: 1.2014 - val_acc: 0.7844
Epoch 4/20
1165/1165 [==============================] - 2s 1ms/step - loss: 0.0248 - acc: 0.9923 - val_loss: 0.8621 - val_acc: 0.8044
Epoch 5/20
1165/1165 [==============================] - 2s 1ms/step - loss: 0.0234 - acc: 0.9923 - val_loss: 0.9717 - val_acc: 0.7946
Epoch 6/20
1165/1165 [==============================] - 2s 1ms/step - loss: 0.0255 - acc: 0.9923 - val_loss: 1.0491 - 

In [0]:
# # cnn_predict = model.predict(data_ts, verbose=0)
# # cnn_predict = np.argmax(cnn_predict,axis=1)

# cnn_image_ts = label_roi_ts
# cnn_image_ts = scipy.ndimage.binary_erosion(cnn_image_ts, structure=np.ones((1,1)))
# cnn_image_ts[cnn_image_ts>0] = cnn_predict

# plt.figure(figsize=(8,8))
# plt.imshow(label_roi_ts, cmap='jet')
# plt.show()


# plt.figure(figsize=(8,8))
# plt.imshow(cnn_image_ts, cmap='jet')
# plt.show()

# # plt.imshow(label_img, cmap='jet')
# # plt.show()
# # plt.figure(figsize=(8,8))
# # plt.imshow(label_roi_tr, cmap='jet')
# # plt.show()
# # plt.figure(figsize=(8,8))
# # plt.imshow(label_roi_ts, cmap='jet')
# # plt.show()

NameError: ignored

In [0]:
# plt.imshow(model.layers[0].get_weights()[0][0,0])
# plt.show()

In [0]:
from keras import backend as K

# with a Sequential model
def get_conv_output(model,data,batch_size=512):
  info = model.get_config()
  flatten_layer = [i['class_name'] for i in info['layers']].index('Flatten')
  get_batch_output = K.function([model.layers[0].input],[model.layers[flatten_layer].output])
  
  n = data.shape[0]
  temp = np.expand_dims(data[0],axis=0)
  out = get_batch_output([temp])[0]
  out_size = out.shape[1]
  ret = np.zeros((n,out_size))
  
  for i in np.arange(0,n,batch_size):
    last = min(i+batch_size,n)
    batch_data = data[i:last]
    if batch_data.ndim == 3:
      batch_data = np.expand_dims(batch_data,axis=0)

    ret[i:last,:] = get_batch_output([batch_data])[0]
  return ret

# Get the convolution outputs as a flattened layer
tr_conv_output = get_conv_output(model,data_tr)
ts_conv_output = get_conv_output(model,data_ts)

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(tr_conv_output)
# tr_conv_output = scaler.transform(tr_conv_output)
# ts_conv_output = scaler.transform(ts_conv_output)



In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data_tr_vector)
dt = scaler.transform(data_tr_vector)
ds = scaler.transform(data_ts_vector)

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import catboost
import time


gap = 1
tr_data = tr_conv_output
ts_data = ts_conv_output
# tr_data = data_tr_vector
# ts_data = data_ts_vector
# tr_data = dt
# ts_data = ds

tr_label = label_tr
ts_label = label_ts

start_time = time.time()
clf_rf = RandomForestClassifier(n_estimators=200,random_state=0)
clf_rf.fit(tr_data[::gap],tr_label[::gap])
score_rf = clf_rf.score(ts_data[::gap],ts_label[::gap])
elapsed_time = time.time() - start_time
print('Random Forest Classifier Accuracy is     {}, elapsed time: {}'.format(score_rf,elapsed_time))

start_time = time.time()
clf_svc = SVC(gamma='scale',random_state=0)
clf_svc.fit(tr_data[::gap],tr_label[::gap])
score_svc = clf_svc.score(ts_data[::gap],ts_label[::gap])
elapsed_time = time.time() - start_time
print('Support Vector Classifier Accuracy is    {}, elapsed_time: {}'.format(score_svc,elapsed_time))

# start_time = time.time()
# clf_cat = catboost.CatBoostClassifier(task_type='GPU',verbose=0,loss_function='MultiClass',random_state=0)
# clf_cat.fit(tr_data[::gap],tr_label[::gap])
# score_cat = clf_cat.score(ts_data[::gap],ts_label[::gap])
# elapsed_time = time.time() - start_time
# print('Catboost Classifier Accuracy is          {}, elapsed_time: {}'.format(score_cat,elapsed_time))

start_time = time.time()
clf_xgb = XGBClassifier()
clf_xgb.fit(tr_data[::gap],tr_label[::gap])
score_xgb = clf_xgb.score(ts_data[::gap],ts_label[::gap])
elapsed_time = time.time() - start_time
print('XGBoost Classifier Accuracy is           {}, elapsed_time: {}'.format(score_xgb,elapsed_time))

start_time = time.time()
clf_gbm = GradientBoostingClassifier(random_state=0)
clf_gbm.fit(tr_data[::gap],tr_label[::gap])
score_gbm = clf_gbm.score(ts_data[::gap],ts_label[::gap])
elapsed_time = time.time() - start_time
print('Gradient Boosting Classifier Accuracy is {}, elapsed_time: {}'.format(score_gbm,elapsed_time))


start_time = time.time()
clf_nb = GaussianNB()
clf_nb.fit(tr_data[::gap],tr_label[::gap])
score_nb = clf_nb.score(ts_data[::gap],ts_label[::gap])
elapsed_time = time.time() - start_time
print('Naive Bayes Classifier Accuracy is      {}, elapsed time: {}'.format(score_nb,elapsed_time))

Random Forest Classifier Accuracy is     0.8080000479385667, elapsed time: 31.61854362487793
Support Vector Classifier Accuracy is    0.8046398540270643, elapsed_time: 187.8006386756897
XGBoost Classifier Accuracy is           0.8100059773400388, elapsed_time: 70.11176538467407
Gradient Boosting Classifier Accuracy is 0.8080914308313896, elapsed_time: 65.21777963638306
Naive Bayes Classifier Accuracy is      0.8158424978390193, elapsed time: 13.755285739898682


In [0]:
gap=5
pred_rf = clf_rf.predict(ts_data[::gap])
pred_svc = clf_svc.predict(ts_data[::gap])
pred_xgb = clf_xgb.predict(ts_data[::gap])
pred_gbm = clf_gbm.predict(ts_data[::gap])
pred_nb = clf_nb.predict(ts_data[::gap])

In [0]:
from scipy.stats import mode

# pred_ens = np.vstack((pred_rf,pred_svc,pred_xgb,pred_gbm))
# sum(label_ts == mode(pred_ens)[0][0])/len(label_ts)

np.corrcoef(pred_rf,pred_nb)

array([[1.        , 0.89772051],
       [0.89772051, 1.        ]])

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import catboost
import time


gap = 1
data_vector_tr = tr_conv_output
data_vector_ts = ts_conv_output
label_vector_tr = label_tr
label_vector_ts = label_ts

start_time = time.time()
clf_rf = RandomForestClassifier(n_estimators=200,random_state=0)
clf_rf.fit(data_vector_tr[::gap],label_vector_tr[::gap])
score_rf = clf_rf.score(data_vector_ts[::gap],label_vector_ts[::gap])
elapsed_time = time.time() - start_time
print('Random Forest Classifier Accuracy is     {}, elapsed time: {}'.format(score_rf,elapsed_time))

start_time = time.time()
clf_svc = SVC(gamma='scale',random_state=0)
clf_svc.fit(data_vector_tr[::gap],label_vector_tr[::gap])
score_svc = clf_svc.score(data_vector_ts[::gap],label_vector_ts[::gap])
elapsed_time = time.time() - start_time
print('Support Vector Classifier Accuracy is    {}, elapsed_time: {}'.format(score_svc,elapsed_time))

start_time = time.time()
clf_cat = catboost.CatBoostClassifier(task_type='GPU',verbose=0,loss_function='MultiClass',random_state=0)
clf_cat.fit(data_vector_tr[::gap],label_vector_tr[::gap])
score_cat = clf_cat.score(data_vector_ts[::gap],label_vector_ts[::gap])
elapsed_time = time.time() - start_time
print('Catboost Classifier Accuracy is          {}, elapsed_time: {}'.format(score_cat,elapsed_time))

start_time = time.time()
clf_xgb = XGBClassifier()
clf_xgb.fit(data_vector_tr[::gap],label_vector_tr[::gap])
score_xgb = clf_xgb.score(data_vector_ts[::gap],label_vector_ts[::gap])
elapsed_time = time.time() - start_time
print('XGBoost Classifier Accuracy is           {}, elapsed_time: {}'.format(score_xgb,elapsed_time))

start_time = time.time()
clf_gbm = GradientBoostingClassifier(random_state=0)
clf_gbm.fit(data_vector_tr[::gap],label_vector_tr[::gap])
score_gbm = clf_gbm.score(data_vector_ts[::gap],label_vector_ts[::gap])
elapsed_time = time.time() - start_time
print('Gradient Boosting Classifier Accuracy is {}, elapsed_time: {}'.format(score_gbm,elapsed_time))

KeyboardInterrupt: ignored

In [0]:
# XgBoost cuda compile

# !git clone https://github.com/dmlc/xgboost --recursive
# !cd xgboost && mkdir build && cd build && cmake .. -DUSE_CUDA=ON && make -j4

-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Setting build type to 'Release' as none was specified.
-- Looking for clock_gettime in rt
-- Looking for clock_gettime in rt - found
-- Looking for fopen64
-- Looking for fopen64 - not found
-- Looking for C++ include cxxabi.h
-- Looking for C++ include

In [0]:
%%time
import lightgbm as lgb
d_train = lgb.Dataset(tr_conv_output[::1], label=label_tr[::1])
params = {}
params['learning_rate'] = 0.1
params['boosting_type'] = 'gbdt'
params['objective'] = 'multiclass'
params['metric'] = 'multi_logloss'
params['feature_fraction'] = 0.9
params['bagging_fraction'] = 0.8
params['bagging_freq'] = 5
params['num_class'] = 11
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 5
# params['device_type'] = 'gpu'
gbm = lgb.train(params, d_train,num_boost_round=30)

CPU times: user 20min 3s, sys: 2.69 s, total: 20min 6s
Wall time: 10min 23s


In [0]:
%%time
pred_gbm = gbm.predict(ts_conv_output, num_iteration=gbm.best_iteration)
pred_gbm_r = np.argmax(pred_gbm,axis=1)
print(np.sum(pred_gbm_r == label_ts)/label_ts.shape[0])

0.7202559702559702
CPU times: user 6.3 s, sys: 9.9 ms, total: 6.31 s
Wall time: 3.27 s


In [0]:
# !pip install pydot
# !pip install pydotplus
# !pip install graphviz
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [0]:
# from __future__ import print_function

# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, BatchNormalization
# from keras.layers import Conv2D, MaxPooling2D
# from keras.optimizers import RMSprop
# from sklearn import preprocessing

# batch_size = 1024
# num_classes = 11
# epochs = 10

# label_tr_cat = keras.utils.to_categorical(label_tr, num_classes)
# label_ts_cat = keras.utils.to_categorical(label_ts, num_classes)

# model = Sequential()
# model.add(BatchNormalization())
# model.add(Conv2D(256, kernel_size=(5, 5),
#                  activation='relu',
#                  input_shape=(16,16,6)))

# model.add(BatchNormalization())
# model.add(Conv2D(512, kernel_size=(3, 3),
#                  activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(1024, kernel_size=(3, 3),
#                  activation='relu'))

# # model.add(Conv2D(128, (3, 3), activation='elu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(BatchNormalization())

# num_layers = 4
# n_neurons = 256
# scaling_factor = 2
# drop_out = 0.1
# drop_out_increment = 0.0125

# for i in np.arange(num_layers):
#   model.add(Dense(n_neurons, activation='relu'))
#   model.add(BatchNormalization())
#   model.add(Dropout(drop_out))
#   n_neurons = int(n_neurons / scaling_factor)
#   drop_out += drop_out_increment
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(data_tr, label_tr_cat,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,validation_data=(data_ts, label_ts_cat))

# # score = model.evaluate(data_ts, label_ts_cat, verbose=0)
# # print('Test loss:', score[0])
# # print('Test accuracy:', score[1])

In [0]:
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.tree import DecisionTreeClassifier


# clf_tree = DecisionTreeClassifier(random_state=0,)
# label_tree = cross_val_predict(clf_tree, data_cat, label_cat, cv=5)
# # print(scores_tree.mean())

# clf_extree = ExtraTreesClassifier(n_estimators=50,n_jobs=-1, random_state=0)
# label_extree = cross_val_score(clf_extree, data_cat, label_cat, cv=5)
# # print(scores_extree.mean())

In [0]:
# # np.log(np.real(data_forest+1e-6))
# # plt.imshow(np.exp(np.real(data_img[:,:,0]*10).astype(float)))
# # plt.show()
# # np.exp(np.real(data_img[:,:,0]*10))

# pred_cnn = model.predict(data_ts,
#           batch_size=batch_size,
#           verbose=1)
# cnn_classes = pred_cnn.argmax(axis=-1)

# import pandas as pd

# df = pd.DataFrame()
# df['label'] = label_ts
# df['CNN'] = cnn_classes

# # df
# pd.crosstab(df['label'],df['CNN'])
